Метод:
1. Получил скорректированные риды. 
2. Выровнял исходные риды на референс, скорректированные парные риды в PE режиме и утратившие парность в single режиме.
3. По всем SAM файлам по CIGAR восстановил выравнивания с вставкой _ на месте инделов.
4. Смёрджил выравнивания исходных ридов и скорректированных по QNAME (предварительно восстановив суффиксы /1, /2 как в исходных парных ридах).
5. Прошёлся по всем параллельным четвёркам нуклеотидов из исходного рида, части референса, куда он выровнялся, скорректированного рида, части референса, куда выровнялся скорректированный рид. Посчитал количество каждых случаев в требумой матрице.

In [1]:
import pandas as pd
import numpy as np
from itertools import zip_longest
import re
from IPython.display import display 
import csv
from multiprocessing import Pool

In [2]:
#SAM_COL_NAMES = ['QNAME', 'FLAG', 'RNAME', 'POS', 'MAPQ', 'CIGAR', 'RNEXT', 'PNEXT', 'TLEN', 'SEQ', 'QUAL', 'TAGS']
SAM_COL_NAMES = ['QNAME', 'POS', 'CIGAR', 'SEQ']

In [3]:
def read_ref(ref_path):
    ref = ''
    with open(ref_path, 'r') as fi:
        for line in fi:
            if line[0] != '>':
                ref += line.strip()
    return ref

def read_sam(sam_path):
    res = []
    with open(sam_path, newline='') as fi:
        reader = csv.DictReader(fi, delimiter='\t', fieldnames=SAM_COL_NAMES, quoting=csv.QUOTE_NONE)
        for row in reader:
            if row['QNAME'][0] != '@':
                res.append(row)
    return res

In [4]:
def parse_cigar(row): 
    read_seq = row['SEQ']
    pos = int(row['POS'])
    cigar = re.findall(r'(\d+)([MIDNSHP=X])', row['CIGAR'])
    seq_aligned = ''
    ref_aligned = ''
    i = 0
    j = pos - 1

    for cig_len_s, cig_ch in cigar:
        cig_len = int(cig_len_s)
        if cig_ch == 'M':
            # match or mismatch
            seq_aligned += read_seq[i:i + cig_len]
            ref_aligned += ref[j:j + cig_len]
        elif cig_ch == 'D':
            # deletion
            seq_aligned += '_' * cig_len
            ref_aligned += ref[j:j + cig_len]
        elif cig_ch == 'I':
            # insertion
            seq_aligned += read_seq[i:i + cig_len]
            ref_aligned += '_' * cig_len
        elif cig_ch == 'S':
            # clipping
            seq_aligned += '_' * cig_len
            ref_aligned += '_' * cig_len
        
        if cig_ch not in 'DNHP':
                i += cig_len
        if cig_ch not in 'ISHP':
                j += cig_len
    
    res = {'read': row['QNAME']}
    res['seq_aligned'] = seq_aligned.replace('N', '_')
    res['ref_aligned'] = ref_aligned
    
    return res

In [5]:
def read_raw_sam(filename_raw):
    sam_raw = read_sam(filename_raw)
    i = 1
    for row in sam_raw:
        row['QNAME'] += f'/{i}'
        i = 3 - i
    sam_raw = [row for row in sam_raw if row['CIGAR'] != '*']
    with Pool() as p:
        res = p.map(parse_cigar, sam_raw)
    return res

In [6]:
def read_spades_sam(filename_paired, filename_unpaired):
    sam_spades_paired = read_sam(filename_paired)
    i = 1
    for row in sam_spades_paired:
        row['QNAME'] += f'/{i}'
        i = 3 - i
    sam_spades_paired = [row for row in sam_spades_paired if row['CIGAR'] != '*']
    with Pool() as p:
        new_sam_spades = p.map(parse_cigar, sam_spades_paired)
    
    sam_spades_unpaired = read_sam(filename_unpaired)
    for row in sam_spades_unpaired:
        row['QNAME'] = '/'.join(row['QNAME'].rsplit('_', 1))
    with Pool() as p:
        res = p.map(parse_cigar, sam_spades_unpaired)
    new_sam_spades.extend(res)
    
    return new_sam_spades

In [7]:
def read_trimmomatic_sam(filename_paired, filename_unpaired1, filename_unpaired2):
    sam_trimmomatic_paired = read_sam(filename_paired)
    i = 1
    for row in sam_trimmomatic_paired:
        row['QNAME'] += f'/{i}'
        i = 3 - i
    sam_trimmomatic_paired = [row for row in sam_trimmomatic_paired if row['CIGAR'] != '*']
    with Pool() as p:
        new_sam_trimmomatic = p.map(parse_cigar, sam_trimmomatic_paired)
    
    sam_trimmomatic_unpaired1 = read_sam(filename_unpaired1)
    for row in sam_trimmomatic_unpaired1:
        row['QNAME'] = '/'.join(row['QNAME'].rsplit('_', 1))
    with Pool() as p:
        res = p.map(parse_cigar, sam_trimmomatic_unpaired1)
    new_sam_trimmomatic.extend(res)
    
    sam_trimmomatic_unpaired2 = read_sam(filename_unpaired2)
    for row in sam_trimmomatic_unpaired2:
        row['QNAME'] = '/'.join(row['QNAME'].rsplit('_', 1))
    with Pool() as p:
        res = p.map(parse_cigar, sam_trimmomatic_unpaired2)
    new_sam_trimmomatic.extend(res)
        
    return new_sam_trimmomatic

In [8]:
def count_row(row):
    count = np.zeros((2, 3), dtype=int)
    
    for raw_seq, raw_ref, corr_seq, corr_ref in zip_longest(row['seq_aligned_raw'], 
                                                    row['ref_aligned_raw'],
                                                    row['seq_aligned_corr'], 
                                                    row['ref_aligned_corr'], 
                                                            fillvalue='_'):

        if raw_seq != raw_ref and corr_seq != corr_ref and raw_seq != '_' and corr_seq != '_':
            count[0, 0] += 1
        elif raw_seq != raw_ref and corr_seq == corr_ref and raw_seq != '_' and corr_seq != '_':
            count[0, 1] += 1
        elif raw_seq != raw_ref and corr_seq == '_':
            count[0, 2] += 1
        elif raw_seq == raw_ref and corr_seq != corr_ref and raw_seq != '_' and corr_seq != '_':
            count[1, 0] += 1
        elif raw_seq == raw_ref and corr_seq == corr_ref and raw_seq != '_' and corr_seq != '_':
            count[1, 1] += 1
        elif raw_seq == raw_ref and corr_seq == '_':
            count[1, 2] += 1
        
    return count

# Тестовые данные

In [9]:
ref = read_ref('MG1655-K12.first10K.fasta')
raw_sam = read_raw_sam('ecoli_10K_err.sam')
raw_sam_df = pd.DataFrame(raw_sam)
raw_sam_df.set_index('read', inplace=True)

## Spades

In [10]:
spades_sam = read_spades_sam('ecoli_10K_spades_paired.sam', 'ecoli_10K_spades_unpaired.sam')

In [11]:
spades_sam_df = pd.DataFrame(spades_sam)
spades_sam_df.set_index('read', inplace=True)

In [12]:
new_sam_df = raw_sam_df.merge(spades_sam_df, left_index=True, right_index=True, how='left', suffixes=('_raw', '_corr')).fillna('')
new_sam = new_sam_df.to_dict(orient='records')

In [13]:
with Pool() as p:
    total_counts = np.zeros((2, 3), dtype=int)
    for count in p.imap_unordered(count_row, new_sam, chunksize=1000):
        total_counts += count

In [14]:
del spades_sam
del spades_sam_df
del new_sam_df
del new_sam

In [15]:
total_counts_df = pd.DataFrame(total_counts, 
            columns=['Error in corrected reads', 'Correct base in corrected reads', 'Base is absent in corrected reads'],
            index=['Error in raw data', 'Correct base in raw data'])

**Абсолютные значения**

In [16]:
total_counts_df

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,2743,16306,33644
Correct base in raw data,16,5272564,595445


**Проценты**

In [17]:
total_counts_df / total_counts.sum() * 100

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,0.046329,0.275406,0.568242
Correct base in raw data,0.000270,89.052780,10.056973


## Trimmomatic

In [18]:
trimmomatic_sam = read_trimmomatic_sam('ecoli_10K_trimmomatic_paired.sam', 
                                       'ecoli_10K_trimmomatic_forward_unpaired.sam', 
                                       'ecoli_10K_trimmomatic_reverse_unpaired.sam')

In [19]:
trimmomatic_sam_df = pd.DataFrame(trimmomatic_sam)
trimmomatic_sam_df.set_index('read', inplace=True)

In [20]:
new_sam_df = raw_sam_df.merge(trimmomatic_sam_df, left_index=True, right_index=True, how='left', suffixes=('_raw', '_corr')).fillna('')
new_sam = new_sam_df.to_dict(orient='records')

In [21]:
with Pool() as p:
    total_counts = np.zeros((2, 3), dtype=int)
    for count in p.imap_unordered(count_row, new_sam, chunksize=1000):
        total_counts += count

In [22]:
del trimmomatic_sam_df
del trimmomatic_sam
del new_sam_df
del new_sam

In [23]:
total_counts_df = pd.DataFrame(total_counts, 
            columns=['Error in corrected reads', 'Correct base in corrected reads', 'Base is absent in corrected reads'],
            index=['Error in raw data', 'Correct base in raw data'])

**Абсолютные значения**

In [24]:
total_counts_df

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,13839,15863,25145
Correct base in raw data,4113,5042099,753822


**Проценты**

In [25]:
total_counts_df / total_counts.sum() * 100

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,0.236367,0.270936,0.429471
Correct base in raw data,0.070249,86.117873,12.875104


# Целые данные

In [26]:
ref = read_ref('MG1655-K12.first400K.fasta')
raw_sam = read_raw_sam('ecoli_400K_err.sam')
raw_sam_df = pd.DataFrame(raw_sam)
raw_sam_df.set_index('read', inplace=True)

## Spades

In [27]:
spades_sam = read_spades_sam('ecoli_400K_spades_paired.sam', 'ecoli_400K_spades_unpaired.sam')

spades_sam_df = pd.DataFrame(spades_sam)
spades_sam_df.set_index('read', inplace=True)

new_sam_df = raw_sam_df.merge(spades_sam_df, left_index=True, right_index=True, how='left', suffixes=('_raw', '_corr')).fillna('')
new_sam = new_sam_df.to_dict(orient='records')

with Pool() as p:
    total_counts = np.zeros((2, 3), dtype=int)
    for count in p.imap_unordered(count_row, new_sam, chunksize=1000):
        total_counts += count

del spades_sam
del spades_sam_df
del new_sam_df
del new_sam

total_counts_df = pd.DataFrame(total_counts, 
            columns=['Error in corrected reads', 'Correct base in corrected reads', 'Base is absent in corrected reads'],
            index=['Error in raw data', 'Correct base in raw data'])

**Абсолютные значения**

In [28]:
total_counts_df

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,210577,717353,1365708
Correct base in raw data,2186,250636102,23046499


**Проценты**

In [29]:
total_counts_df / total_counts.sum() * 100

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,0.076302,0.259931,0.494860
Correct base in raw data,0.000792,90.817281,8.350834


## Trimmomatic

In [30]:
trimmomatic_sam = read_trimmomatic_sam('ecoli_400K_trimmomatic_paired.sam', 
                                       'ecoli_400K_trimmomatic_forward_unpaired.sam', 
                                       'ecoli_400K_trimmomatic_reverse_unpaired.sam')

trimmomatic_sam_df = pd.DataFrame(trimmomatic_sam)
trimmomatic_sam_df.set_index('read', inplace=True)

new_sam_df = raw_sam_df.merge(trimmomatic_sam_df, left_index=True, right_index=True, how='left', suffixes=('_raw', '_corr')).fillna('')
new_sam = new_sam_df.to_dict(orient='records')

with Pool() as p:
    counts = p.map(count_row, new_sam)

del trimmomatic_sam_df
del trimmomatic_sam
del new_sam_df
del new_sam

total_counts = sum(counts)
total_counts_df = pd.DataFrame(total_counts, 
            columns=['Error in corrected reads', 'Correct base in corrected reads', 'Base is absent in corrected reads'],
            index=['Error in raw data', 'Correct base in raw data'])

**Абсолютные значения**

In [31]:
total_counts_df

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,693737,698188,1012642
Correct base in raw data,200099,243532522,28018019


**Проценты**

In [32]:
total_counts_df / total_counts.sum() * 100

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,0.253045,0.254669,0.369368
Correct base in raw data,0.072987,88.830165,10.219765
